In [ ]:
import ee
import geemap
import numpy
import requests
import io
import pandas as pd
import geopandas as gpd
import json

# Run if need new data

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-isekalala')

# Load Village Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
villages_dir = '/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/Lamwo_villages_2020.gpkg'

In [ ]:
village_gdf = gpd.read_file(villages_dir)

In [ ]:
village_gdf.head()

,ID,V_OBJECTID,Shape_Leng,Shape_Area,addr_dname,addr_cname,addr_sname,addr_pname,addr_vname,addr_rname,REA_PROJEC,NRECA_MP,geometry
0,5500895,32615,1540.032051,1.485227e+05,Lamwo,Lamwo,Agoro,Ngacino,Special Area,Northern,F,F,"MULTIPOLYGON (((33.0218 3.80975, 33.02515 3.80..."
1,5500896,32616,42482.085300,8.355897e+07,Lamwo,Lamwo,Agoro,Ngacino,Grazing Land,Northern,F,F,"MULTIPOLYGON (((32.9609 3.84951, 32.97234 3.85..."
2,5500897,32617,6762.484374,2.436672e+06,Lamwo,Lamwo,Agoro,Ngacino,Irumo,Northern,F,F,"MULTIPOLYGON (((33.0019 3.8277, 33.00515 3.830..."
3,5500898,32618,7285.491885,2.806164e+06,Lamwo,Lamwo,Agoro,Ngacino,Obere,Northern,F,F,"MULTIPOLYGON (((33.01002 3.81807, 33.01014 3.8..."
4,5500899,32619,6189.132451,2.041819e+06,Lamwo,Lamwo,Agoro,Ngacino,Kila,Northern,F,F,"MULTIPOLYGON (((33.02373 3.81576, 33.02527 3.8..."


# Run if new data is needed

In [ ]:
# Assume `gdf` is your GeoDataFrame and `index` is the row you want to convert
index = 0  # Adjust the index to the specific row you want
geometry = village_gdf.iloc[index].geometry

def geom_to_ee_fmt(geometry):
  # Convert the geometry to a GeoJSON format
  geometry_geojson = geometry.__geo_interface__

# Ensure it is in the format expected by ee.Geometry.MultiPolygon
  if geometry_geojson["type"] == "MultiPolygon":
    # Create an ee.Geometry.MultiPolygon
    ee_geometry = ee.Geometry.MultiPolygon(geometry_geojson["coordinates"])
  elif geometry_geojson["type"] == "Polygon":
    # If it's a single Polygon, convert it to MultiPolygon
    ee_geometry = ee.Geometry.MultiPolygon([geometry_geojson["coordinates"]])
  else:
    raise ValueError("Geometry type is not Polygon or MultiPolygon.")
  return ee_geometry


# ee_geometry is now an Earth Engine MultiPolygon geometry
village = geom_to_ee_fmt(geometry)
print(village.getInfo())

{'type': 'Polygon', 'coordinates': [[[33.021795932856996, 3.8097489636418054], [33.0214095313367, 3.806644512528437], [33.02301920233525, 3.8062564133735934], [33.02485427551908, 3.8075498874955143], [33.02623122133802, 3.8084275816661677], [33.025151252073, 3.809956278325074], [33.021795932856996, 3.8097489636418054]]]}


In [ ]:
def get_open_buildings_counts(region, year, scale_m=1):
  imageCollection = ee.ImageCollection('GOOGLE/Research/open-buildings-temporal/v1')
  yr = str(year)
  mosaic = imageCollection.filterBounds(region).filterDate(yr + '-01-01', yr + '-12-31').mosaic()

  count = mosaic.reduceRegion(ee.Reducer.sum(), region, scale_m, crs= region.projection(), maxPixels=824814450) \
          .getNumber('building_fractional_count') \
          .multiply(ee.Number(scale_m * 2).pow(2))
          # Since the pyramiding policy is mean, we need to multiply by
          # (scale_m * 2) ** 2 to recover sum at original 50cm resolution.
  return int(count.getInfo())

In [ ]:
# prompt: use the ID column from df to make a new dataframe with ID as the primary key, and 8 columns labeled 2016 to 2023 respectively. The ID's are used to load the respective polygons, and having their building counts calculated for the respective column

new_df = pd.DataFrame(columns=['ID', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])

for i in range(len(village_gdf)):
  village = village_gdf.iloc[i]
  try:
    villagegeom = geom_to_ee_fmt(village.geometry)
    village_id = village.ID
    row_data = {'ID': village_id}
    for year in range(2016, 2024):
      try:
        building_count = get_open_buildings_counts(villagegeom, year)
        row_data[str(year)] = building_count
      except:
        row_data[str(year)] = None
    new_df = pd.concat([new_df, pd.DataFrame([row_data])], ignore_index=True)
  except Exception as e:
    print(f"Error processing village {village_id}: {e}")

new_df = new_df.set_index('ID')
print(new_df)


In [ ]:
# prompt: find rows in new_df with null values in their feature columns

# Find rows with null values in feature columns
null_rows = new_df[new_df.isnull().any(axis=1)]
null_rows

# Load historic building counts for villages, and merge with village dataset

In [ ]:
new_df = pd.read_csv('/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/Lamwo_villages_building_counts.csv')

In [ ]:
# prompt: join new df with df using the ID column, sort by the column '2023', descending. drop columns V_OBJECTID, Shape_Leng, Shape_Area, addr_cname

# Assuming 'df' and 'new_df' are already defined as in your provided code.

merged_df = pd.merge(village_gdf, new_df, on='ID', how='left')

# Sort by '2023' column in descending order
merged_df = merged_df.sort_values(by='2023', ascending=False)

# Drop specified columns
columns_to_drop = ['V_OBJECTID', 'Shape_Leng', 'Shape_Area', 'addr_cname', 'geometry']
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

merged_df.head()

,ID,addr_dname,addr_sname,addr_pname,addr_vname,addr_rname,REA_PROJEC,NRECA_MP,2016,2017,2018,2019,2020,2021,2022,2023
122,5502921,Lamwo,Palabek Ogili,Apyetta,Apyetta South,Northern,F,T,453,1766,4144,4537,4895,4745,3874,5036
121,5502920,Lamwo,Palabek Ogili,Apyetta,Apyetta Central,Northern,F,T,643,1658,3823,4095,4051,4080,3113,4121
123,5502922,Lamwo,Palabek Gem,Anaka,Aja Ogala,Northern,F,F,519,906,3178,3936,4335,4356,3248,4108
106,5502905,Lamwo,Palabek Ogili,Lugwar,Lagotopuk,Northern,F,T,983,971,1846,2817,3051,3137,2438,2870
65,5502861,Lamwo,Palabek Gem,Anaka,Beyogoya,Northern,F,F,220,221,710,873,1017,1178,973,1686


# Analyses

In [ ]:
merged_df

,ID,addr_dname,addr_sname,addr_pname,addr_vname,addr_rname,REA_PROJEC,NRECA_MP,2016,2017,2018,2019,2020,2021,2022,2023
122,5502921,Lamwo,Palabek Ogili,Apyetta,Apyetta South,Northern,F,T,453,1766,4144,4537,4895,4745,3874,5036
121,5502920,Lamwo,Palabek Ogili,Apyetta,Apyetta Central,Northern,F,T,643,1658,3823,4095,4051,4080,3113,4121
123,5502922,Lamwo,Palabek Gem,Anaka,Aja Ogala,Northern,F,F,519,906,3178,3936,4335,4356,3248,4108
106,5502905,Lamwo,Palabek Ogili,Lugwar,Lagotopuk,Northern,F,T,983,971,1846,2817,3051,3137,2438,2870
65,5502861,Lamwo,Palabek Gem,Anaka,Beyogoya,Northern,F,F,220,221,710,873,1017,1178,973,1686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,5501328,Lamwo,Lamwo Tc,Ateng,Awac,Northern,F,F,0,0,0,0,0,0,0,0
164,5502963,Lamwo,Padibe East,Wangtit,Ogako North,Northern,F,F,0,0,0,0,0,0,0,0
169,5502968,Lamwo,Padibe East,Wangtit,Locken West,Northern,F,F,0,0,0,0,0,0,0,0
348,5503149,Lamwo,Madi Opei,Madi Opei Town Board,Central,Northern,F,F,0,0,0,0,0,0,0,0


In [ ]:
# prompt: for the merged_df, perform linear regression for each row using the columns 2016-2023, so we can predict the number of 2024 and 2025

import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming 'merged_df' is already defined and contains the relevant columns.

# Define the years for prediction
years_to_predict = [2024, 2025]

#make a copy of merged_df with our new columns
village_counts_predictive = merged_df.copy()

# Create empty columns for the predicted values
for year in years_to_predict:
  village_counts_predictive[str(year)] = None

# Iterate through each row in the DataFrame
for index, row in merged_df.iterrows():
  # Extract the features (years 2016-2023)
  X = []
  y = []
  for year in range(2016, 2024):
    value = row[str(year)]
    if pd.notna(value):  # Check if value is not NaN
        X.append([year])
        y.append(value)

  # Check if we have enough valid data points for regression
  if len(X) >= 2 : # Need at least two data points to perform regression
    # Create and train a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict the values for 2024 and 2025
    for year in years_to_predict:
        predicted_value = model.predict([[year]])
        village_counts_predictive.loc[index, str(year)] = predicted_value[0]
  else:
    print(f"Not enough valid data points for row {index} to perform linear regression")

#all negative values in the columns [2024, 2025] become zero
for year in years_to_predict:
  village_counts_predictive[str(year)] = village_counts_predictive[str(year)].clip(lower=0)


village_counts_predictive


,ID,addr_dname,addr_sname,addr_pname,addr_vname,addr_rname,REA_PROJEC,NRECA_MP,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
122,5502921,Lamwo,Palabek Ogili,Apyetta,Apyetta South,Northern,F,T,453,1766,4144,4537,4895,4745,3874,5036,6080.285714,6613.404762
121,5502920,Lamwo,Palabek Ogili,Apyetta,Apyetta Central,Northern,F,T,643,1658,3823,4095,4051,4080,3113,4121,4930.928571,5316.02381
123,5502922,Lamwo,Palabek Gem,Anaka,Aja Ogala,Northern,F,F,519,906,3178,3936,4335,4356,3248,4108,5257.142857,5742.452381
106,5502905,Lamwo,Palabek Ogili,Lugwar,Lagotopuk,Northern,F,T,983,971,1846,2817,3051,3137,2438,2870,3584.714286,3878.178571
65,5502861,Lamwo,Palabek Gem,Anaka,Beyogoya,Northern,F,F,220,221,710,873,1017,1178,973,1686,1693.857143,1879.214286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,5501328,Lamwo,Lamwo Tc,Ateng,Awac,Northern,F,F,0,0,0,0,0,0,0,0,0.0,0.0
164,5502963,Lamwo,Padibe East,Wangtit,Ogako North,Northern,F,F,0,0,0,0,0,0,0,0,0.0,0.0
169,5502968,Lamwo,Padibe East,Wangtit,Locken West,Northern,F,F,0,0,0,0,0,0,0,0,0.0,0.0
348,5503149,Lamwo,Madi Opei,Madi Opei Town Board,Central,Northern,F,F,0,0,0,0,0,0,0,0,0.0,0.0


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=village_counts_predictive)

https://docs.google.com/spreadsheets/d/1emPrDR-QjN8DidN3i6qZKzXI2RliUQjprrHK3GJfb_M#gid=0


In [ ]:
# prompt: which 20 villages have seen the most significant increase in building count from 2016 - 2023?

# Calculate the increase in building count from 2016 to 2023
merged_df['increase'] = merged_df['2023'] - merged_df['2016']

# Sort by the increase in descending order and get the top 5
top_20_villages = merged_df.sort_values(by='increase', ascending=False).head(20)

# Display the top 5 villages
print(top_20_villages[['ID', 'addr_vname', '2016', '2023', 'increase']])

          ID        addr_vname  2016  2023  increase
122  5502921     Apyetta South   453  5036      4583
123  5502922         Aja Ogala   519  4108      3589
121  5502920   Apyetta Central   643  4121      3478
106  5502905         Lagotopuk   983  2870      1887
65   5502861          Beyogoya   220  1686      1466
382  5503183    Akworo Central   266  1587      1321
102  5502901       Akworo East   109  1358      1249
187  5502988      Grazing Land   110   688       578
63   5502859          Beyogoya   194   673       479
108  5502907    Mudu Northwest    18   455       437
19   5501315         Ngomoromo   893  1288       395
120  5502919    Alimotiko West   474   765       291
50   5502037            Agwata   537   790       253
258  5503059        Ywaya West   212   416       204
266  5503067     Lela Bul East   129   285       156
93   5502892  Padwat Northwest   327   469       142
250  5503051              Apuk   154   296       142
109  5502908         Mudu West   107   244    

In [ ]:
top_20_villages[['ID', 'addr_vname', '2016', '2023', 'increase']].to_csv('/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/Lamwo_villages_building_increase_top_20.csv')

In [ ]:
villages_existing_grids = pd.read_csv('/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/villages_with_existing_minigrids.csv')

In [ ]:
# prompt: select the rows in merged_df that have ID in villages_existing_grids

# Select rows where 'ID' is present in villages_existing_grids
existing_mgs_building_increases = merged_df[merged_df['ID'].isin(villages_existing_grids['ID'])]

#sort by 'increase' descending
existing_mgs_building_increases.sort_values(by='increase', ascending=False)

NameError: name 'merged_df' is not defined

In [ ]:
existing_mgs_building_increases.to_csv('/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/lamwo_villages_building_change_for_existing_mgs.csv')

In [ ]:
candidate_villages = pd.read_csv('/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/candidate villages/candidate_villages.csv')

In [ ]:
# prompt: join new df with df using the ID column, sort by the column '2023', descending. drop columns V_OBJECTID, Shape_Leng, Shape_Area, addr_cname

# Assuming 'df' and 'new_df' are already defined as in your provided code.

merged_df = pd.merge(candidate_villages, new_df, on='ID', how='left')

# Sort by '2023' column in descending order
merged_df = merged_df.sort_values(by='2023', ascending=False)

# Drop specified columns
columns_to_drop = ['V_OBJECTID', 'Shape_Leng', 'Shape_Area', 'addr_cname']
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

merged_df.head()

,ID,addr_dname,addr_sname,addr_pname,addr_vname,addr_rname,REA_PROJEC,NRECA_MP,2016,2017,2018,2019,2020,2021,2022,2023
12,5502921,Lamwo,Palabek Ogili,Apyetta,Apyetta South,Northern,F,T,453,1766,4144,4537,4895,4745,3874,5036
13,5502922,Lamwo,Palabek Gem,Anaka,Aja Ogala,Northern,F,F,519,906,3178,3936,4335,4356,3248,4108
24,5503134,Lamwo,Madi Opei,Pobura,Pobutu,Northern,F,T,669,725,606,718,662,650,652,642
10,5502906,Lamwo,Palabek Ogili,Lugwar,Lugwar Central,Northern,F,T,430,390,435,475,490,533,485,562
2,5502858,Lamwo,Palabek Gem,Gem,Medde South,Northern,F,F,527,513,513,532,451,527,465,514


In [ ]:
# prompt: which 5 candidate villages have seen the most significant increase in building count from 2016 - 2023?

# Calculate the increase in building count from 2016 to 2023
merged_df['increase'] = merged_df['2023'] - merged_df['2016']

# Sort by the increase in descending order and get the top 5
top_5_villages = merged_df.sort_values(by='increase', ascending=False).head(5)

# Display the top 5 villages
print(top_5_villages[['ID', 'addr_vname', '2016', '2023', 'increase']])

         ID        addr_vname 2016  2023 increase
12  5502921     Apyetta South  453  5036     4583
13  5502922         Aja Ogala  519  4108     3589
5   5502892  Padwat Northwest  327   469      142
1   5502849          Guruguru  363   498      135
10  5502906    Lugwar Central  430   562      132


In [ ]:
top_5_villages

,ID,addr_dname,addr_sname,addr_pname,addr_vname,addr_rname,REA_PROJEC,NRECA_MP,2016,2017,2018,2019,2020,2021,2022,2023,increase
12,5502921,Lamwo,Palabek Ogili,Apyetta,Apyetta South,Northern,F,T,453,1766,4144,4537,4895,4745,3874,5036,4583
13,5502922,Lamwo,Palabek Gem,Anaka,Aja Ogala,Northern,F,F,519,906,3178,3936,4335,4356,3248,4108,3589
5,5502892,Lamwo,Palabek Ogili,Padwat,Padwat Northwest,Northern,F,F,327,299,299,317,370,446,396,469,142
1,5502849,Lamwo,Palabek Kal,Kal,Guruguru,Northern,F,T,363,343,338,366,412,424,376,498,135
10,5502906,Lamwo,Palabek Ogili,Lugwar,Lugwar Central,Northern,F,T,430,390,435,475,490,533,485,562,132
